In [3]:
from rsplib import RSPEngine, RSPPublisher, Stream, rdf_table, load_graph, accessURL

In [4]:
jasper = RSPEngine("http://jasper:8181/jasper")

In [27]:
qid  = 'reasoning'
tbox = 'https://linkeddata.stream/ontologies/colors.owl'
frmt = 'JSON'
body =  ''' 
        PREFIX colors: <http://www.streamreasoning.org/ontologies/2018/9/colors#>
        SELECT (COUNT(?warm) AS ?countWarm) (COUNT(?cold) AS ?countCold)
        FROM NAMED WINDOW <colorwindow> ON <http://colorstream:8080/colors> [RANGE PT15S STEP PT5S]
        WHERE {
            { WINDOW <colorwindow> { ?warm a colors:Warm . } }
            UNION
            { WINDOW <colorwindow> { ?cold a colors:Cold .} }
        }
    '''

In [28]:
reasoning1q = jasper.create(qid, body, tbox, frmt)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [29]:
jasper.streams()

[http://jasper:8181/jasper/streams/green,
 http://jasper:8181/jasper/streams/reasoning,
 http://jasper:8181/jasper/streams/bluecount,
 http://colorstream:8080/colors]

In [8]:
r = jasper.expose(qid, 'HTTP', retention=5)

In [9]:
reasoning1_stream = reasoning1q.stream()

In [22]:
r.endpoints()

[GET http://jasper:8184/jasper/streams/reasoning/observers/696118539]

In [23]:
endpoint = reasoning1_stream.endpoints()[0]

In [24]:
df=endpoint.call().dataframe()

In [25]:
df

,countCold,countWarm,eventTime,processingTime
0,0,1,1592762534654,1592762534658
1,0,1,1592762534654,1592762534660
2,0,1,1592762535055,1592762535057
3,0,1,1592762535055,1592762535058
4,0,1,1592762535055,1592762535059


In [26]:
%matplotlib inline
import pandas as pd

df1 = df[['cntWarm','cntCold']]
cntg_mean = df1['cntCold'].mean()
cntr_mean = df1['cntWarm'].mean()

df = pd.DataFrame({'means': [cntg_mean, cntr_mean]}, index=['Cold', 'Warm'])
plot = df.plot.pie(y='means', colors=["#0000e1", "#e10000"], figsize=(5, 5))

KeyError: "['cntWarm' 'cntCold'] not in index"

In [55]:
jasper.delete(qid)

{
    "@id": "queries/reasoning",
    "@type": "vprov:Task",
    "prov:generated": {
        "@id": "streams/reasoning"
    },
    "prov:uses": {
        "@id": "http://streamhub:9292/streamhub/streams/colorstream"
    },
    "@context": {
        "@base": "http://host.docker.internal:9999/jasper/",
        "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
        "vsd": "http://w3id.org/rsp/vocals-sd#",
        "vocals": "http://w3id.org/rsp/vocals#",
        "xsd": "http://www.w3.org/2001/XMLSchema#",
        "format": "http://www.w3.org/ns/formats/",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "vprov": "http://w3id.org/rsp/vocals-prov#",
        "dcat": "http://www.w3.org/ns/dcat#",
        "prov": "http://www.w3.org/ns/prov#"
    }
}